# Brain Tumor Classification: Manual Gridsearch
The data has already been preprocessed in the Setup & EDA notebook.  I took the hard route on this project, that being a manual grid search over hyperparameters for a model trained from scratch.  One great alternative to a manaul gridsearch is to *wrap the keras model in an sklearn wrapper object and use an sklearn GridSearch or RandomSearch object.*  The reason I'm skipping this option is two-fold:  
1. this approach isn't compatible with Tensorboard and I wanted to explore my results with TensorBoard this time, 
2. To distrubute this workload across multiple instances, stop, start, and pick up another time, it's easiest to make a list of hyperparameter combinations and track your index in this list, or split the list up between instances such as those on Colab  

The second part to this, the fact I'm training a model from scratch instead of applying transfer learning, is simply to see how well I can do for this project and then compare the results to what I can achieve with a basic transfer learning application.


<h1>Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Brain-Tumor-Classification:-Manual-Gridsearch" data-toc-modified-id="Brain-Tumor-Classification:-Manual-Gridsearch-1">Brain Tumor Classification: Manual Gridsearch</a></span></li><li><span><a href="#import" data-toc-modified-id="import-2">import</a></span></li><li><span><a href="#paths" data-toc-modified-id="paths-3">paths</a></span></li><li><span><a href="#reproducibility-&amp;-maintenance" data-toc-modified-id="reproducibility-&amp;-maintenance-4">reproducibility &amp; maintenance</a></span></li><li><span><a href="#data-generators" data-toc-modified-id="data-generators-5">data generators</a></span></li><li><span><a href="#classes" data-toc-modified-id="classes-6">classes</a></span></li><li><span><a href="#gridsearch" data-toc-modified-id="gridsearch-7">gridsearch</a></span><ul class="toc-item"><li><span><a href="#hyperparams" data-toc-modified-id="hyperparams-7.1">hyperparams</a></span></li><li><span><a href="#create_model()" data-toc-modified-id="create_model()-7.2">create_model()</a></span></li><li><span><a href="#train_test_model()" data-toc-modified-id="train_test_model()-7.3">train_test_model()</a></span></li><li><span><a href="#pickle_bm_lists()" data-toc-modified-id="pickle_bm_lists()-7.4">pickle_bm_lists()</a></span></li><li><span><a href="#run_benchmarking()" data-toc-modified-id="run_benchmarking()-7.5">run_benchmarking()</a></span></li><li><span><a href="#run-it!" data-toc-modified-id="run-it!-7.6">run it!</a></span></li></ul></li><li><span><a href="#Time-to-examine-the-results!" data-toc-modified-id="Time-to-examine-the-results!-8">Time to examine the results!</a></span></li></ul></div>

# import

In [1]:
# tensorflow tools #
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
from tensorflow.keras.layers import (Dense, Input, MaxPooling2D, MaxPool2D, GlobalAveragePooling2D,
                                     Conv2D, Flatten, Dropout, BatchNormalization) 
from tensorflow.keras.activations import sigmoid, softmax, relu
from tensorflow.keras.optimizers import RMSprop, Adam, Adagrad, SGD
from tensorflow.keras.utils import to_categorical
# sklearn wrapper 
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# ml viz
from tensorboard.plugins.hparams import api as hp 
%load_ext tensorboard
import pydot 
import graphviz 
# sklearn tools #
import sklearn 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA, TruncatedSVD
# image tools # 
from tensorflow.keras.preprocessing import image 
import cv2
import imutils
# data handling tools #
import numpy as np
import pandas as pd 
# plotting tools #
import matplotlib.pyplot as plt 
import seaborn as sns 
# general tools #
import datetime
from collections import Counter
import os 
pjoin = os.path.join
import shutil
import gc
import pickle 
import itertools
import re 
import string 
# import kaggle

# paths

In [5]:
train_path, val_path, test_path = './train', './val', './test'
model_path = './model'
bm_lists_path = './bm_lists'
tb_log_path = './tb_runlog'

path_ls = [train_path, val_path, test_path, model_path, 
           bm_lists_path, tb_log_path]

for path in path_ls:
    if not os.path.exists(path):
        os.mkdir(path)

# reproducibility & maintenance

In [6]:
keras.backend.clear_session()
tf.random.set_seed(38)
np.random.seed(38)
_= gc.collect() 

# data generators

In [16]:
# train generator instantiation 
train_im_datagen = image.ImageDataGenerator(rescale=1./255, 
                                     rotation_range=.15, 
                                     width_shift_range=0, 
                                     height_shift_range=0, 
                                     brightness_range=(.1, .9),
                                     shear_range=.15,
                                     zoom_range=0,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     data_format='channels_last',
                                     validation_split=0)
# test generator instantiation
test_im_datagen = image.ImageDataGenerator(rescale=1./255)

In [17]:
def get_data_gens(batch_size):
    ''' Given the batch size, returns a tuple of generators from the 
        designated train, val, and test paths:  (traingen, valgen, testgen). '''
    # train data flow inititalized #
    traingen = train_im_datagen.flow_from_directory(train_path, 
                                                    target_size=target_size, 
                                                    color_mode='rgb',
                                                    class_mode='categorical', 
                                                    batch_size=batch_size, 
                                                    shuffle=True,
                                                    seed=38)
    # val data flow inititalized #
    valgen = test_im_datagen.flow_from_directory(val_path, 
                                                 target_size=target_size, 
                                                 color_mode='rgb',
                                                 class_mode='categorical', 
                                                 batch_size=batch_size, 
                                                 shuffle=True,
                                                 seed=38)
    # test data flow inititalized #
    testgen = test_im_datagen.flow_from_directory(test_path, 
                                                  target_size=target_size, 
                                                  color_mode='rgb',
                                                  class_mode='categorical', 
                                                  batch_size=batch_size, 
                                                  shuffle=True,
                                                  seed=38)
    
    return traingen, valgen, testgen

# classes

In [19]:
class Clock():
    ''' A simple clock class that prints or hands back the elapsed time between 
        start and stop calls in a human friendly format. '''
    import datetime
    def __init__(self):
        self.running = False
        self.start_time = None
        self.stop_time = None
        self.elapsed = None
        
    def start(self):
        self.running = True            
        self.start_time = datetime.datetime.now()
        
    def stop(self, stdout=True, handback=False):
        if self.running:
            self.running = False
            self.end_time = datetime.datetime.now()
            self.delta = str(self.end_time - self.start_time).split(':')
            self.delta[2] = self.delta[2][:2]
#             self.elapsed = 'hours:{0[0]}, minutes:{0[1]}, seconds:{0[2]}'\
#                                                             .format(self.delta)
            self.elapsed = 'minutes:{0[1]}, seconds:{0[2]}'.format(self.delta)
            if stdout:
                print(self.elapsed)
            if handback:
                return self.elapsed
            
    def __repr__(self):
        if self.running:
            return 'The clock is running!'
        else:
            return 'The clock is not running.'

# gridsearch

In [24]:
keras.backend.clear_session()
_= gc.collect() 

## hyperparams

In [44]:
epochs = 20
target_size = (224,224)
classes_dim = 4

In [2]:
conv_layers = [2, 4]
filters = [32, 128]
kernel_size = [3, 7]    # conv window size
pool_size = [2]
dense_hidden_layers = [0, 1]
optimizer = ['adam', 'rmsprop']
lr = [.001, .01]
do_rate = [0., .5]
batch_normalize = [True, False]
batch_size = [64, 128]

hyperparams = list(itertools.product(conv_layers, filters, kernel_size, pool_size, 
                                     dense_hidden_layers, optimizer, lr, do_rate, 
                                     batch_normalize, batch_size))
param_names = \
'''conv_layers, filters, kernel_size, pool_size, dense_hidden_layers, optimizer, \
lr, do_rate, batch_normalize, batch_size'''.split(', ')

In [3]:
len(hyperparams)

512

## create_model()

In [47]:
def create_model(name, hparams):
    ''' Recieves a name for the model and the hyperparameters to create it and returns the 
        compiled model. '''
    # optimizer #
    if hparams['optimizer'] == 'adam':
        optimizer = Adam(learning_rate=hparams['lr'])
    elif hparams['optimizer'] == 'rmsprop':
        optimizer = RMSprop(learning_rate=hparams['lr'], momentum=.9)
    # bias vector #    
    if hparams['batch_normalize'] == True:
        use_bias = False
    else:
        use_bias = True 

    # *******  input block  ******* #
    input_ = Input(shape=(*target_size, 3), name='input')

    # *******  conv blocks  ******* #
    # conv block 1 #
    x = Conv2D(filters=hparams['filters'], kernel_size=hparams['kernel_size'], strides=1, 
               padding='same', use_bias=use_bias, activation='relu')(input_)
    if hparams['batch_normalize'] == True:
            x = BatchNormalization()(x)
    x = MaxPool2D(pool_size=hparams['pool_size'], strides=hparams['pool_size'])(x)
    # conv blocks 2 --> #
    for i in range(1, hparams['conv_layers']):
        x = Conv2D(filters=hparams['filters'], kernel_size=hparams['kernel_size'], strides=1, 
               padding='same', use_bias=use_bias, activation='relu')(x)
        if hparams['batch_normalize'] == True:
                x = BatchNormalization()(x)
        x = MaxPool2D(pool_size=hparams['pool_size'], strides=hparams['pool_size'])(x)
    
    # *******  dense blocks  ******* #
    x = Flatten()(x)
    x = Dropout(rate=hparams['do_rate'])(x)
    # dense blocks 1 --> #
    for i in range(0, hparams['dense_hidden_layers']):
        x = Dense(units=hparams['filters'], activation='relu', use_bias=use_bias)(x)
        if hparams['batch_normalize'] == True:
                x = BatchNormalization()(x)       
    # output #
    output_ = Dense(units=classes_dim, activation='softmax', name='output')(x)

    
    # *******  model  ******* #
    m = Model(inputs=[input_], outputs=[output_], name=name)

    # *******  compile  ******* #
    m.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    
    print(m.summary())
    
    return m    

## train_test_model()

In [48]:
def train_test_model(m:tf.keras.Model, fname, hparams, epochs):
    ''' Trains the given model, evaluates it on the test data, and logs the results with the 
    tensorboard callback in the fname location. Returns the history dict and the test accuracy. '''
    #***********#   callbacks   #***********#
    # cb for logging the metrics
    tb_callback = keras.callbacks.TensorBoard(log_dir=fname, histogram_freq=1, 
                                              write_images=True)
    # cb for logging the parameters
    hp_callback = hp.KerasCallback(fname, hparams)
    # cb for early stopping
    earlystop_callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=.01, 
                                                       baseline=.25, patience=5)
    
    #***********#   data gens   #***********#
    # data #
    traingen, valgen, testgen = get_data_gens(hparams['batch_size'])
    
    #***********#   fit and evaluate   #***********#
    # fit #
    h = m.fit(traingen, 
              steps_per_epoch=(traingen.n // traingen.batch_size),
              validation_data=valgen, 
              validation_steps=(valgen.n // valgen.batch_size),
              epochs=epochs, 
              callbacks=[tb_callback, hp_callback, earlystop_callback])

    # evaluate on test data #
    test_loss, test_acc = m.evaluate(testgen)

    return h.history, test_acc

## pickle_bm_lists()

In [49]:
def pickle_bm_lists(bm_ls_mapper):
    for path,ls in bm_ls_mapper.items():
        with open(pjoin(bm_lists_path, path), 'wb') as file: 
            pickle.dump(ls, file)

## run_benchmarking()

In [50]:
def run_benchmarking():
    ''' For the designated indices, the hyperparameter space is explored. All relevant benchmarks
        are stored in lists and pickled in the current directory for safe keeping. '''
    # maintenance #
    keras.backend.clear_session() 
    gc.collect()
    !rm -rf tb_log_path
    # benchark lists #
    history_ls, test_acc_ls, time_ls, param_ls = [], [], [], []
    # benchmark lists fname mapper
    bm_ls_mapper = {'history_ls.pkl':history_ls, 'test_acc_ls.pkl':test_acc_ls, 
                    'time_ls.pkl':time_ls, 'param_ls.pkl':param_ls}
    # this is where distribution across multiple instances can be orchestrated #
    start_idx, end_idx = 0, len(hyperparams)
    
    #******************************** run it ********************************#
    for run in range(start_idx, end_idx):
        params = hyperparams[run]
        param_dict = dict(zip(param_names, params))
        print(f'\n\nRUN {run}\n{param_dict}')

        # create model #
        m = create_model(str(run), param_dict)

        # train and test the model #
        clock.start()
        h, test_acc = train_test_model(m, 
                                       os.path.join(tb_log_path, 'run_', str(run)), 
                                       param_dict,
                                       epochs)
        clock.stop() 
        
        # append current benchmarks to appropriate bm list
        param_ls.append(param_dict)        
        history_ls.append(h)
        test_acc_ls.append(test_acc)
        time_ls.append(clock.elapsed)
        
        # save lists #
        if run % 10 == 0:
            pickle_bm_lists(bm_ls_mapper)

    # save lists #
    pickle_bm_lists(bm_ls_mapper)
    
    return 'finished' 

## run it!

In [4]:
runtime_clock = Clock()
runtime_clock.start()

### ******************* ###
run_benchmarking()
### ******************* ###

runtime_clock.stop() 

# Time to examine the results!